In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import LSTM,Dropout,GRU,MaxPooling1D,Flatten,concatenate
from keras.models import Sequential
from keras.layers import Dense,Concatenate,Bidirectional
from keras.layers.convolutional import Conv1D
import tensorflow as tf
from keras.utils.vis_utils import model_to_dot
from keras.layers import RepeatVector
from keras.layers import TimeDistributed,Activation
from sklearn.preprocessing import MinMaxScaler
from keras import Input,Model
import io
FD1_train = pd.read_csv("train_FD001.txt",delimiter=" ",header = None)
FD1_test = pd.read_csv("test_FD001.txt",delimiter=" ",header = None)
RUL_file=pd.read_csv("RUL_FD001.txt",header = None)

FD1_test=FD1_test.drop([26, 27], axis = 1)
columns = {0:'unit_number',1:'time_in_cycles',2:'opSetting1',3:'opSetting2',4:'opSetting3',5:'sensor1',6:'sensor2',
           7:'sensor3',8:'sensor4',9:'sensor5',10:'sensor6',11:'sensor7',12:'sensor8',13:'sensor9',14:'sensor10',
           15:'sensor11',16:'sensor12',17:'sensor13',18:'sensor14',19:'sensor15',20:'sensor16',
           21:'sensor17',22:'sensor18',23:'sensor19',24:'sensor20',25:'sensor21'}
FD1_test = FD1_test.rename(columns=columns)
FD1_test=FD1_test.drop(columns = ['opSetting1','opSetting2','opSetting3','sensor1','sensor5','sensor6','sensor10','sensor16','sensor18','sensor19','sensor17'], axis =1)

cols_normalize = FD1_test.columns.difference(
    ['unit_number', 'time_in_cycles'])  

min_max_scaler = MinMaxScaler()

norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(FD1_test[cols_normalize]),
                             columns=cols_normalize,
                             index=FD1_test.index)



join_df = FD1_test[FD1_test.columns.difference(cols_normalize)].join(norm_train_df)
FD1_test = join_df.reindex(columns=FD1_test.columns)

FD1_test=FD1_test.drop(['time_in_cycles'],axis=1)
data_cols= FD1_test.columns

def prepare_test_data(df):
    data_list = []
    for unit_number in df.unit_number.unique():
        unit = df[df.unit_number == unit_number]
        data_list.append(np.array(unit[data_cols])[-31:, :])
    return np.stack(data_list)
FD1_test=prepare_test_data(FD1_test)
FD1_test=FD1_test[:,:,1:]

print(FD1_test.shape)







FD1_train=FD1_train.drop([26, 27], axis = 1)
columns = {0:'unit_number',1:'time_in_cycles',2:'opSetting1',3:'opSetting2',4:'opSetting3',5:'sensor1',6:'sensor2',
           7:'sensor3',8:'sensor4',9:'sensor5',10:'sensor6',11:'sensor7',12:'sensor8',13:'sensor9',14:'sensor10',
           15:'sensor11',16:'sensor12',17:'sensor13',18:'sensor14',19:'sensor15',20:'sensor16',
           21:'sensor17',22:'sensor18',23:'sensor19',24:'sensor20',25:'sensor21'}
FD1_train = FD1_train.rename(columns=columns)
FD1_train=FD1_train.drop(columns = ['opSetting1','opSetting2','opSetting3','sensor1','sensor5','sensor6','sensor10','sensor16','sensor18','sensor19','sensor17'], axis =1)
def Calcolo_RUL(df):
    df_copy=df.copy()
    gruppo=df_copy.groupby('unit_number')['time_in_cycles'].max().reset_index()
    gruppo.columns = ['unit_number','max_cycles_fu']
    df_copy = df_copy.merge(gruppo, on=['unit_number'], how='left')
    df['RUL'] = df_copy['max_cycles_fu'] - df_copy['time_in_cycles']
    return df
Calcolo_RUL(FD1_train)
rul_clip_limit = 120
FD1_train=FD1_train.clip(upper=rul_clip_limit)



cols_normalize = FD1_train.columns.difference(
    ['unit_number', 'time_in_cycles', 'RUL'])  

min_max_scaler = MinMaxScaler()

norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(FD1_train[cols_normalize]),
                             columns=cols_normalize,
                             index=FD1_train.index)


join_df = FD1_train[FD1_train.columns.difference(cols_normalize)].join(norm_train_df)
FD1_train = join_df.reindex(columns=FD1_train.columns)
print(FD1_train.info)
FD1_c=FD1_train.copy()
FD1_train = FD1_train.drop(['RUL','time_in_cycles'], axis=1)

def gen_sequence(id_df, seq_length, seq_cols):
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

seq_gen = (list(gen_sequence(FD1_train[FD1_train['unit_number']==unit_number],31, FD1_train.columns))
           for unit_number in FD1_train['unit_number'].unique())

seq_array = np.concatenate(list(seq_gen)).astype(np.float32)

seq_array=seq_array[:,:,1:]

print(seq_array.shape)
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

label_gen = [gen_labels(FD1_c[FD1_c['unit_number']==id], 31, ['RUL'])
             for id in FD1_c['unit_number'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)



model= Sequential()
model.add(Conv1D(128,1,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2,padding='same',strides=2))
model.add(Conv1D(64,1,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2,padding='same',strides=2))
model.add(Conv1D(32,1,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2,padding='same',strides=2))
model.add(LSTM(32, input_shape=(31, 14), return_sequences=True))
model.add(LSTM(32, input_shape=(31, 14), return_sequences=False))
model.add(Flatten())
model.add(Dense(128,kernel_initializer='uniform', activation="relu"))
model.add(Dense(30,kernel_initializer='uniform', activation="relu"))
model.add(Dense(1,kernel_initializer='uniform', activation="relu"))

model.compile(loss='mean_squared_error', optimizer='adam',metrics = [tf.keras.metrics.RootMeanSquaredError()])
model.fit(seq_array,label_array ,batch_size=100,epochs=50,verbose=1,validation_split=0.1)
y_pred= model.predict(FD1_test)
from sklearn.metrics import mean_squared_error
from math import sqrt
RUL_file=pd.read_csv("RUL_FD001.txt",header = None)
print(sqrt(mean_squared_error(RUL_file, y_pred)))
print(y_pred)
print((mean_squared_error(RUL_file, y_pred)))

(100, 31, 13)
<bound method DataFrame.info of        unit_number  time_in_cycles  sensor2  ...  sensor20  sensor21  RUL
0                1               1      0.0  ...  0.713178  0.724662  120
1                1               2      0.0  ...  0.666667  0.731014  120
2                1               3      0.0  ...  0.627907  0.621375  120
3                1               4      0.0  ...  0.573643  0.662386  120
4                1               5      0.0  ...  0.589147  0.704502  120
...            ...             ...      ...  ...       ...       ...  ...
20626          100             120      0.0  ...  0.271318  0.109500    4
20627          100             120      0.0  ...  0.124031  0.366197    3
20628          100             120      0.0  ...  0.232558  0.053991    2
20629          100             120      0.0  ...  0.116279  0.234466    1
20630          100             120      0.0  ...  0.178295  0.218172    0

[20631 rows x 16 columns]>
(17531, 31, 13)
Epoch 1/50
158/158 [==